<a href="https://colab.research.google.com/github/soxjasonshm/JasonPoonLab01/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install opencv-python

In [4]:
!unzip archive.zip
archive_dir = '/content/archive.zip'

Archive:  archive.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory
dataset_dir = '/content/train'

# Set the desired image and batch sizes
image_size = (48, 48)
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load and preprocess the training set
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    classes=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
)

# Load and preprocess the validation set
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    classes=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the trained model
model.save('emotions_model.h5')

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Epoch 1/10
718/718 [==============================] - 43s 53ms/step - loss: 1.7939 - accuracy: 0.2520 - val_loss: 1.7643 - val_accuracy: 0.2743
Epoch 2/10
718/718 [==============================] - 37s 52ms/step - loss: 1.7402 - accuracy: 0.2834 - val_loss: 1.7121 - val_accuracy: 0.3005
Epoch 3/10
718/718 [==============================] - 38s 52ms/step - loss: 1.6682 - accuracy: 0.3294 - val_loss: 1.6235 - val_accuracy: 0.3520
Epoch 4/10
718/718 [==============================] - 37s 52ms/step - loss: 1.5746 - accuracy: 0.3817 - val_loss: 1.5372 - val_accuracy: 0.4031
Epoch 5/10
718/718 [==============================] - 37s 52ms/step - loss: 1.5016 - accuracy: 0.4161 - val_loss: 1.5089 - val_accuracy: 0.4173
Epoch 6/10
718/718 [==============================] - 37s 52ms/step - loss: 1.4491 - accuracy: 0.4390 - val_loss: 1.4456 - val_accuracy: 0.4431
Epoch 7/10
718/718 [==============================]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load pre-trained model
model = tf.keras.models.load_model('/content/emotions_model.h5')

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load face detection cascade (Haar cascade)
face_cascade = cv2.CascadeClassifier('path/to/haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(0)

# Continuously process frames
while True:
    # Read frame from video capture
    ret, frame = cap.read()

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract face region
        face = gray[y:y+h, x:x+w]

        # Resize face to match input size of the model
        resized_face = cv2.resize(face, (48, 48))

        # Normalize pixel values
        normalized_face = resized_face / 255.0

        # Reshape face for model input
        input_face = np.expand_dims(normalized_face, axis=0)
        input_face = np.expand_dims(input_face, axis=-1)

        # Perform emotion recognition
        predictions = model.predict(input_face)
        emotion_index = np.argmax(predictions[0])
        emotion_label = emotion_labels[emotion_index]

        # Draw bounding box and emotion label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Real-time Emotion Recognition', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()